In [ ]:
import argparse
import numpy as np
from get_data import main
from cnn.training import get_trained_model

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', help='Path to the config file', default='./default_config.yml')
parser.add_argument('--log-dir', help='Path to the log directory', default='../log/')
parser.add_argument('--posters', help='Path to the posters', default='../data/posters/')
parser.add_argument('--models-dir', help='Path to the saved models', default='../data/models/')
parser.add_argument('--sets-dir', help='Path to the training and testing sets', default='../data/sets/')
parser.add_argument('--database', help='Path to the databse csv', default='../data/poster_data.csv')
parser.add_argument('--csv', help='Path to the clean csv', default='../data/')
parser.add_argument('-s', '--save', help='Save model', action='store_true')
parser.add_argument('-v', '--verbose', help='Verbose', action='store_true')

str_args = '-v -s'.split()
args, _ = parser.parse_known_args(str_args)

In [ ]:
clean_movies, train_posters, train_genres, train_ids, test_posters, test_genres, test_ids, model_name, save_model, verbose = main(args)

In [ ]:
resnet, _ = get_trained_model(model_name)

In [ ]:
train_features_resnet = resnet.predict(train_posters).reshape((len(train_posters), -1))
test_features_resnet = resnet.predict(test_posters).reshape((len(test_posters), -1))

# PCA

In [ ]:
from sklearn.decomposition import PCA

pca_resnet = PCA(n_components=0.98)

In [ ]:
from sklearn.preprocessing import StandardScaler

SC = StandardScaler()
SC.fit(train_features_resnet)
train_features_resnet_stand = SC.transform(train_features_resnet)
test_features_resnet_stand = SC.transform(test_features_resnet)

In [ ]:
pca_resnet.fit(train_features_resnet_stand)

In [ ]:
# train_resnet_pca = pca_resnet.transform(train_features_resnet)
# test_resnet_pca = pca_resnet.transform(test_features_resnet)

train_resnet_pca = pca_resnet.transform(train_features_resnet_stand)
test_resnet_pca = pca_resnet.transform(test_features_resnet_stand)

In [ ]:
print(train_features_resnet.shape[1], train_resnet_pca.shape[1])

# KNN 

In [ ]:
from knn.knn import KNN

k = 7
knn = KNN(k)
knn.fit(train_features_resnet, train_genres)

In [ ]:
from utils.accuracy import mono_label

predicted_genres = knn.predict(test_features_resnet, train_genres)
mono_label(test_genres, predicted_genres)

# XGBoost

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

train_genres_flat = np.array([np.nonzero(x)[0] for x in train_genres]).squeeze()
boost = XGBClassifier(objective='multi:softmax', num_classes=7)
boost.fit(train_resnet_pca, train_genres_flat)

In [ ]:
predicted_genres = boost.predict(test_resnet_pca)
test_genres_flat = np.array([np.nonzero(x)[0] for x in test_genres]).squeeze()

In [ ]:
# boost.save_model('current.model')
np.mean(test_genres_flat == predicted_genres)

In [ ]:
plt.bar(range(len(boost.feature_importances_)), boost.feature_importances_)
plt.show()

# SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier # Regarder les autres multiclass possibles
# J'ai testé vite fait OneVsOne: pb de dimension, autre format d'input attendu

svc = OneVsRestClassifier(SVC(gamma='auto'))
svc.fit(train_resnet_pca, train_genres)

In [ ]:
from utils.accuracy import mono_label

predicted_genres = svc.predict(test_resnet_pca)
mono_label(test_genres, predicted_genres)

# Analyse des résultats

In [ ]:
import yaml

config = yaml.safe_load(open(args.config))
class_names = {config['genres'][k]: k for k in config['genres'].keys()}

In [ ]:
from utils.display.results_display import ConfusionMatrix_display

title = 'ResNet+kNN, k='+str(k) 
ConfusionMatrix_display(test_genres, predicted_genres, config['genres'], title)

In [ ]:
from utils.display.results_display import histogram

histogram(test_genres, predicted_genres, k, config['genres'])

In [ ]:
from utils.display.results_display import plot_neighbors
%matplotlib inline

starting_index = np.random.randint(0, 700)
num_images = 5
plot_neighbors(test_posters, test_genres, class_names, predicted_genres, starting_index, num_images, train_posters, train_genres, knn.neighbors,str(starting_index))

Affichage de la matrice de confusion

## PCA 2D

In [ ]:
import matplotlib.pyplot as plt

def display2D(X_ortho, Y):
    colors = Y.argmax(axis=1)
    
    fig = plt.figure(figsize=(10, 10))
    scatter = plt.scatter(X_ortho[:, 0], X_ortho[:, 1], c=colors)
    plt.legend(*scatter.legend_elements())
    
    plt.show(fig)

In [ ]:
pca = PCA(n_components=2)
# pca.fit(train_posters.reshape(len(train_posters), -1))

train_pca = pca.transform(train_posters.reshape(len(train_posters), -1))
# test_pca = pca.transform(test_posters.reshape(len(train_posters), -1))

display2D(train_pca, train_genres)
display2D(train_resnet_pca[:, :2], train_genres)

#Attention: l'erreur dans la VM vient de la version de matplotlib; et upgrade vers matplotlib 3.1.X pas possible
#Changer version Python...